In [1]:
import pandas as pd
import matplotlib as plt
df=pd.read_excel('DB_detallado.xlsx')

In [2]:
columnas_resum=['fecha','CO_IPRESS','CA_CONSULTORIOS', 'CA_CONSULTORIOS_FN', 'CA_CAMAS','CA_MEDICOS_TOTAL','CA_ENFERMERAS',
 'CA_ODONTOLOGOS','CA_PSICOLOGOS','CA_NUTRICIONISTAS','CA_TECNOLOGOS_MEDICOS', 'CA_OBSTETRICES','CA_FARMACEUTICOS','CA_AUXILIARES','CA_OTROS_PROF','CA_AMBULANCIAS','CA_MEDICOS_SERUM','CA_MEDICOS_RESIDENTES']
df['fecha'] = pd.to_datetime(df['ANHO'].astype(str) + '-' + df['MES'].astype(str), format='%Y-%m')

concat_sumable=df[columnas_resum]
latest_entries = concat_sumable.groupby(['CO_IPRESS', concat_sumable['fecha'].dt.year]).apply(lambda x: x.sort_values('fecha').tail(1)).reset_index(drop=True)
latest_entries['ANHO']=latest_entries['fecha'].dt.year
latest_entries

columnas_resum=['fecha','CA_CONSULTORIOS', 'CA_CONSULTORIOS_FN', 'CA_CAMAS','CA_MEDICOS_TOTAL','CA_MEDICOS_SERUM','CA_MEDICOS_RESIDENTES','CA_ENFERMERAS',
 'CA_ODONTOLOGOS','CA_PSICOLOGOS','CA_NUTRICIONISTAS','CA_TECNOLOGOS_MEDICOS', 'CA_OBSTETRICES','CA_FARMACEUTICOS','CA_AUXILIARES','CA_OTROS_PROF','CA_AMBULANCIAS','ID_PROCEDIMIENTO',
'UBIGEO', 'DEPARTAMENTO','PROVINCIA','DISTRITO','SECTOR','CATEGORIA','RAZON_SOC','MES']
selected_columns = df[[col for col in df.columns if col not in columnas_resum]]
selected_columns=selected_columns.groupby(['CO_IPRESS','ANHO']).sum().reset_index()

selected_columns

merged_df = selected_columns.merge(latest_entries, on=['CO_IPRESS','ANHO'])
merged_df

,CO_IPRESS,ANHO,Partos_CESAREA,Partos_VAGINAL,Partos_vivos_CESAREA,Partos_vivos_VAGINAL,Partos_muertos_CESAREA,Partos_muertos_VAGINAL,DE_PROGRAMC,TOTAL_CIRUJ_MAY,...,CA_PSICOLOGOS,CA_NUTRICIONISTAS,CA_TECNOLOGOS_MEDICOS,CA_OBSTETRICES,CA_FARMACEUTICOS,CA_AUXILIARES,CA_OTROS_PROF,CA_AMBULANCIAS,CA_MEDICOS_SERUM,CA_MEDICOS_RESIDENTES
0,1,2017,1014.0,2045.0,1015.0,2050.0,11.0,0.0,33.0,1990.0,...,2.0,3.0,1.0,33.0,6.0,252.0,5.0,1.0,0.0,18.0
1,1,2018,1041.0,2192.0,1029.0,2222.0,27.0,1.0,33.0,2204.0,...,5.0,3.0,1.0,36.0,14.0,232.0,8.0,1.0,0.0,12.0
2,1,2019,1012.0,2345.0,982.0,2341.0,42.0,0.0,36.0,2445.0,...,5.0,3.0,1.0,36.0,14.0,232.0,8.0,1.0,0.0,12.0
3,1,2020,715.0,2647.0,672.0,2647.0,43.0,0.0,33.0,2585.0,...,5.0,2.0,1.0,33.0,5.0,208.0,7.0,1.0,0.0,10.0
4,1,2021,1231.0,2795.0,1202.0,2793.0,29.0,2.0,36.0,3033.0,...,6.0,7.0,1.0,29.0,11.0,275.0,6.0,1.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16317,30708,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
16318,30727,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0
16319,31275,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,1.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0
16320,31276,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0


In [3]:
data_hosp=pd.read_csv('Bases de Apoyo\\IPRESS.csv',encoding='ISO-8859-1',sep=',')
data_hosp = data_hosp.rename(columns={'Código Único': 'CO_IPRESS'})
depxIPRESS=data_hosp[['CO_IPRESS','Departamento']]

depResm=merged_df.merge(depxIPRESS,on='CO_IPRESS')
depResm.drop(['CO_IPRESS','fecha'], inplace=True,axis=1)

depResm=depResm.groupby(['ANHO','Departamento']).sum().reset_index()

In [4]:
depResm.rename(columns={'Departamento': 'DEPARTAMENTO'},inplace=True)

In [5]:
presupuesto_2017 = pd.read_excel('Bases de Apoyo\\Presupuestos.xlsx',sheet_name='2017')
presupuesto_2018 = pd.read_excel('Bases de Apoyo\\Presupuestos.xlsx',sheet_name='2018')
presupuesto_2019 = pd.read_excel('Bases de Apoyo\\Presupuestos.xlsx',sheet_name='2019')
presupuesto_2020 = pd.read_excel('Bases de Apoyo\\Presupuestos.xlsx',sheet_name='2020')
presupuesto_2021 = pd.read_excel('Bases de Apoyo\\Presupuestos.xlsx',sheet_name='2021')
presupuesto_2022 = pd.read_excel('Bases de Apoyo\\Presupuestos.xlsx',sheet_name='2022')

In [6]:
df_2017 = pd.merge(depResm.loc[depResm['ANHO']==2017],presupuesto_2017,on='DEPARTAMENTO',how='inner')
df_2018 = pd.merge(depResm.loc[depResm['ANHO']==2018],presupuesto_2018,on='DEPARTAMENTO',how='inner')
df_2019 = pd.merge(depResm.loc[depResm['ANHO']==2019],presupuesto_2019,on='DEPARTAMENTO',how='inner')
df_2020 = pd.merge(depResm.loc[depResm['ANHO']==2020],presupuesto_2020,on='DEPARTAMENTO',how='inner')
df_2021 = pd.merge(depResm.loc[depResm['ANHO']==2021],presupuesto_2021,on='DEPARTAMENTO',how='inner')
df_2022 = pd.merge(depResm.loc[depResm['ANHO']==2022],presupuesto_2022,on='DEPARTAMENTO',how='inner')
df_apoyo = pd.concat([df_2017,df_2018,df_2019,df_2020,df_2021,df_2022],axis=0)
df_apoyo

,ANHO,DEPARTAMENTO,Partos_CESAREA,Partos_VAGINAL,Partos_vivos_CESAREA,Partos_vivos_VAGINAL,Partos_muertos_CESAREA,Partos_muertos_VAGINAL,DE_PROGRAMC,TOTAL_CIRUJ_MAY,...,CA_TECNOLOGOS_MEDICOS,CA_OBSTETRICES,CA_FARMACEUTICOS,CA_AUXILIARES,CA_OTROS_PROF,CA_AMBULANCIAS,CA_MEDICOS_SERUM,CA_MEDICOS_RESIDENTES,PIA,PIM
0,2017,AMAZONAS,1253.0,2587.0,1252.0,2427.0,11.0,37.0,144.0,3524.0,...,15.0,84.0,11.0,241.0,51.0,15.0,0.0,14.0,108797360.0,170888254.0
1,2017,ANCASH,3580.0,7580.0,3622.0,7483.0,33.0,169.0,267.0,9802.0,...,62.0,638.0,45.0,1878.0,159.0,117.0,148.0,39.0,226188435.0,361961579.0
2,2017,APURIMAC,1463.0,2531.0,1480.0,2478.0,6.0,36.0,186.0,4323.0,...,12.0,271.0,39.0,631.0,505.0,45.0,67.0,18.0,167848815.0,282011418.0
3,2017,AREQUIPA,4614.0,6289.0,4660.0,6256.0,44.0,41.0,216.0,20828.0,...,14.0,130.0,49.0,1094.0,134.0,24.0,19.0,109.0,301635031.0,547114468.0
4,2017,AYACUCHO,2609.0,4697.0,2597.0,5184.0,48.0,72.0,327.0,6571.0,...,7.0,405.0,75.0,684.0,628.0,61.0,38.0,16.0,217519248.0,394854270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,2022,PUNO,2692.0,5820.0,2716.0,5759.0,43.0,80.0,331.0,7118.0,...,27.0,231.0,43.0,921.0,380.0,29.0,2.0,62.0,566512834.0,824713379.0
21,2022,SAN MARTIN,4389.0,6506.0,4435.0,6501.0,109.0,47.0,1053.0,12587.0,...,87.0,478.0,71.0,2028.0,588.0,60.0,60.0,13.0,434186618.0,642831246.0
22,2022,TACNA,1352.0,1205.0,1369.0,1196.0,8.0,19.0,35.0,737.0,...,12.0,60.0,9.0,627.0,69.0,4.0,0.0,25.0,184434832.0,266532998.0
23,2022,TUMBES,1275.0,2103.0,1278.0,2098.0,8.0,6.0,36.0,2617.0,...,22.0,157.0,4.0,579.0,6.0,24.0,50.0,10.0,123586953.0,184525893.0


In [7]:
df_apoyo2=df_apoyo[['ANHO','DEPARTAMENTO']]#separamos otras columnas que se volveran a añadir despues de reescalar
df_apoyo.drop(columns=['ANHO','DEPARTAMENTO'],inplace=True)

In [8]:
df_apoyo['Partos_Vivos']=df_apoyo['Partos_vivos_CESAREA']+df_apoyo['Partos_vivos_VAGINAL']#Resumimos la información en una sola columna y eliminamos el resto
df_apoyo['Partos_Muertos']=df_apoyo['Partos_muertos_CESAREA']+df_apoyo['Partos_muertos_VAGINAL']
df_apoyo.drop(columns=['Partos_CESAREA','Partos_VAGINAL','Partos_vivos_CESAREA', 'Partos_vivos_VAGINAL', 'Partos_muertos_CESAREA','Partos_muertos_VAGINAL'],inplace=True)


In [9]:
df_apoyo['Total_cirugia']=df_apoyo['TOTAL_CIRUJ_MAY']+df_apoyo['TOTAL_CIRUJ_MEN']
df_apoyo.drop(columns=['TOTAL_CIRUJ_MAY','TOTAL_CIRUJ_MEN','DE_PROGRAMC','HORAS_PROGRAMADAS','HORAS_ACT_QUIRURJ'],inplace=True)
df_apoyo.drop(columns=['CA_MEDICOS_SERUM','CA_MEDICOS_RESIDENTES'],inplace=True) #la columna medicos total ya refleja la suma de ambas columnas

In [10]:
df_apoyo['Total_Infecciones_Intrahospitalarias']=df_apoyo['Numero de Casos Nuevos  de Infeccion Intrahospitalaria en Neonatos']+df_apoyo['Numero de Casos Nuevos de  Infeccion Intrahospitalaria en Gineco obstetricia']+df_apoyo['Numero de Casos Nuevos de  Infeccion Intrahospitalaria en UCI']+df_apoyo['Numero de Casos Nuevos de Infección del tracto urinario por cateter']+df_apoyo['Numero de Casos Nuevos de infeccion de herida operatoria']
df_apoyo.drop(columns=['Numero de Casos Nuevos  de Infeccion Intrahospitalaria en Neonatos','Numero de Casos Nuevos de  Infeccion Intrahospitalaria en Gineco obstetricia','Numero de Casos Nuevos de  Infeccion Intrahospitalaria en UCI','Numero de Casos Nuevos de Infección del tracto urinario por cateter','Numero de Casos Nuevos de infeccion de herida operatoria'],inplace=True) #la columna medicos total ya refleja la suma de ambas columnas

In [11]:
df_apoyo.drop(columns=['Partos en Gestantes que han recibido por lo menos un control prenatal en la IPRESS','Partos en gestante que han recibido la primera consulta prenatal en el primer trimestre del embarazo', 'Partos que se producen en gestantes que han recibido por lo menos 6 controles prenatales.'],inplace=True) #estas columnas ya son reflejadas por la columna partos total
df_apoyo.drop(columns=['NRO_TOTAL_PACIENTES_CAMAS','NRO_TOTAL_CAMAS','CA_CAMAS'],inplace=True)

In [12]:
df_apoyo.dropna(inplace=True)

A continuación reescalamos los atributos, se utilizaron dos formas distintas para este fin.

In [13]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
ss=MinMaxScaler()
reescalar=[]
for i in df_apoyo:
    if ((df_apoyo[i].dtype) in ['int64','float']):
        reescalar.append(str(i))

df_MinMax=pd.DataFrame(ss.fit_transform(df_apoyo[reescalar]),columns=reescalar)
df_Log=np.log1p(df_apoyo[reescalar])

Eliminamos outliers

In [16]:
q1=df_MinMax.quantile(0.25)
q3=df_MinMax.quantile(0.75)
RIQ=q3-q1
sup=q3+1.5*RIQ
inf=q1-1.5*RIQ


In [17]:
df_MinMax.shape

(150, 40)

In [18]:
mascara=(((df_MinMax<= inf)|(df_MinMax>= sup)).sum(axis=1))>0
mascara
df_apoyo3=df_MinMax[~mascara]
df_apoyo3.shape

(72, 40)

**PIA Y EL PIM**

In [19]:
y=df_apoyo3.PIM
k=10 #Se optimizo este hiperpárametro para seleccionar la menor cantidad de atributos que nos permitan alcanzar un mayor coeficiente de determinación

In [20]:
X=df_apoyo3.drop(columns=['PIA','PIM'],inplace=False)#No se aceptan variables objetos, las columnas no pueden ser numeros tampoco


## Seleccion de atributos

### 1.Filtro usando f_regression

A continuación se muestran los atributos seleccionados utilizando un filtro

In [21]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
fit_selector = SelectKBest(score_func=f_regression, k=k).fit(X, y)
print("Atributos Seleccionados (por posición): {}".format(fit_selector.get_support(indices=True)))
print('Lista de atributos seleccionados')
print(list(X.columns[fit_selector.get_support(indices=True)]))



Atributos Seleccionados (por posición): [ 2  3  4  5  6 23 24 27 34 35]
Lista de atributos seleccionados
['NRO_TOTAL_HOSPIT_ING', 'NRO_TOTAL_HOSPIT_EGR', 'NRO_TOTAL_ESTANCIAS', 'NRO_TOTAL_CAMAS_DISPONIB', 'NRO_TOTAL_FALLECIDOS', 'CA_MEDICOS_TOTAL', 'CA_ENFERMERAS', 'CA_NUTRICIONISTAS', 'Partos_Vivos', 'Partos_Muertos']


Para la tabla 1

In [22]:
df_apoyo=pd.DataFrame({'Scores de los atributos':fit_selector.scores_,'P value':fit_selector.pvalues_})
(df_apoyo.set_index(fit_selector.feature_names_in_).sort_values(by='Scores de los atributos',ascending=False)).head(15)

,Scores de los atributos,P value
NRO_TOTAL_CAMAS_DISPONIB,87.536173,5.900074e-14
Partos_Vivos,78.702713,4.550528e-13
Partos_Muertos,72.210913,2.213688e-12
CA_MEDICOS_TOTAL,64.885640,1.445751e-11
NRO_TOTAL_FALLECIDOS,54.537081,2.469949e-10
CA_ENFERMERAS,52.145107,4.928103e-10
NRO_TOTAL_ESTANCIAS,51.457122,6.026926e-10
NRO_TOTAL_HOSPIT_EGR,50.948815,6.998637e-10
NRO_TOTAL_HOSPIT_ING,44.564350,4.847869e-09
CA_NUTRICIONISTAS,36.455404,6.717231e-08


In [23]:
X_filtrada_f = pd.DataFrame(fit_selector.transform(X),columns=X.columns[fit_selector.get_support(indices=True)])
X_filtrada_f


,NRO_TOTAL_HOSPIT_ING,NRO_TOTAL_HOSPIT_EGR,NRO_TOTAL_ESTANCIAS,NRO_TOTAL_CAMAS_DISPONIB,NRO_TOTAL_FALLECIDOS,CA_MEDICOS_TOTAL,CA_ENFERMERAS,CA_NUTRICIONISTAS,Partos_Vivos,Partos_Muertos
0,0.032218,0.203173,0.070665,0.125101,0.072151,0.042377,0.076187,0.052174,0.221082,0.094378
1,0.108687,0.667578,0.319091,0.494275,0.238062,0.259939,0.417631,0.495652,0.671990,0.403614
2,0.050882,0.305026,0.161338,0.286596,0.104915,0.117519,0.251695,0.173913,0.238023,0.082329
3,0.114991,0.715371,0.297765,0.367613,0.176020,0.128877,0.185082,0.095652,0.675937,0.351406
4,0.131091,0.767564,0.507094,0.643077,0.285117,0.182176,0.218588,0.104348,0.691845,0.447791
...,...,...,...,...,...,...,...,...,...,...
67,0.101217,0.608852,0.351882,0.445121,0.240153,0.247270,0.264858,0.200000,0.581213,0.186747
68,0.087439,0.573479,0.295545,0.607806,0.188567,0.168196,0.268049,0.391304,0.512296,0.244980
69,0.120917,0.763032,0.384131,0.511959,0.171488,0.266929,0.336657,0.286957,0.661728,0.311245
70,0.030329,0.175554,0.114994,0.196580,0.123736,0.076453,0.115676,0.052174,0.153440,0.052209


### 2. Wrapper: Selección de atributos recursiva

In [24]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

modelo = LinearRegression()

recursivefe = RFE(modelo, n_features_to_select=k, step=2) #en cada iteración se eliminan dos columnas
fit_selector = recursivefe.fit(X, y)

print("Numero de Atributos: {}".format(fit_selector.n_features_))
print("Atributos Seleccionados (por posición): {}".format(fit_selector.get_support(indices=True)))
print('Lista de atributos seleccionados')
print(list(X.columns[fit_selector.get_support(indices=True)]))



Numero de Atributos: 10
Atributos Seleccionados (por posición): [ 8  9 11 17 18 21 23 25 26 29]
Lista de atributos seleccionados
['Casos Nuevos de Gestantes con diagnostico confirmado de Tuberculosis', 'Casos Nuevos de Gestantes con diagnostico confirmado de infección HIV.', 'Casos Nuevos de TBC pulmonar en el personal del EESS', 'Niños de 37 meses que completaron esquema nacional de inmunizaciones', 'Numero de Controles Prenatales realizados.', 'CA_CONSULTORIOS', 'CA_MEDICOS_TOTAL', 'CA_ODONTOLOGOS', 'CA_PSICOLOGOS', 'CA_OBSTETRICES']


In [25]:
# Data reducida
X_filtrada_RFE = pd.DataFrame(fit_selector.transform(X),columns=X.columns[fit_selector.get_support(indices=True)])
X_filtrada_RFE

,Casos Nuevos de Gestantes con diagnostico confirmado de Tuberculosis,Casos Nuevos de Gestantes con diagnostico confirmado de infección HIV.,Casos Nuevos de TBC pulmonar en el personal del EESS,Niños de 37 meses que completaron esquema nacional de inmunizaciones,Numero de Controles Prenatales realizados.,CA_CONSULTORIOS,CA_MEDICOS_TOTAL,CA_ODONTOLOGOS,CA_PSICOLOGOS,CA_OBSTETRICES
0,0.000000,0.002801,0.000000,0.139478,0.071424,0.016710,0.042377,0.052632,0.071066,0.071979
1,0.010791,0.008403,0.004604,0.500294,0.156641,0.536311,0.259939,0.563910,0.294416,0.546701
2,0.000000,0.002801,0.000000,0.000000,0.002577,0.289846,0.117519,0.394737,0.213198,0.232219
3,0.000000,0.000000,0.000000,0.079470,0.085530,0.048843,0.128877,0.078947,0.065990,0.124250
4,0.017986,0.016807,0.000512,0.022701,0.033717,0.043380,0.182176,0.082707,0.106599,0.132819
...,...,...,...,...,...,...,...,...,...,...
67,0.000000,0.002801,0.000000,0.026637,0.099273,0.023458,0.247270,0.041353,0.091371,0.134533
68,0.000000,0.000000,0.000000,0.011884,0.049955,0.049486,0.168196,0.169173,0.147208,0.197943
69,0.000000,0.030812,0.005627,0.108572,0.311414,0.227828,0.266929,0.296992,0.380711,0.409597
70,0.000000,0.000000,0.000000,0.011001,0.012924,0.022815,0.076453,0.026316,0.065990,0.051414


### 3.Wrapper: Selección de atributos secuencial

In [26]:
from sklearn.feature_selection import SequentialFeatureSelector

sfs = SequentialFeatureSelector(modelo, n_features_to_select=k)#tambien se emplea un decision tree regressor
fit_selector = sfs.fit(X, y)

print("Atributos Seleccionados: {}".format(sfs.get_support(indices=True)))
print('Lista de atributos seleccionados:')
print(list(X.columns[sfs.get_support(indices=True)]))


Atributos Seleccionados: [ 7  8 12 13 17 18 21 23 24 29]
Lista de atributos seleccionados:
['TOTAL', 'Casos Nuevos de Gestantes con diagnostico confirmado de Tuberculosis', 'Casos nuevos de infeccion HIV.', 'Fallecimientos en todos los servicios.', 'Niños de 37 meses que completaron esquema nacional de inmunizaciones', 'Numero de Controles Prenatales realizados.', 'CA_CONSULTORIOS', 'CA_MEDICOS_TOTAL', 'CA_ENFERMERAS', 'CA_OBSTETRICES']


In [27]:
#Data reducida
X_filtrada_SFE = pd.DataFrame(sfs.transform(X),columns=X.columns[sfs.get_support(indices=True)])
X_filtrada_SFE

,TOTAL,Casos Nuevos de Gestantes con diagnostico confirmado de Tuberculosis,Casos nuevos de infeccion HIV.,Fallecimientos en todos los servicios.,Niños de 37 meses que completaron esquema nacional de inmunizaciones,Numero de Controles Prenatales realizados.,CA_CONSULTORIOS,CA_MEDICOS_TOTAL,CA_ENFERMERAS,CA_OBSTETRICES
0,0.016983,0.000000,0.047273,0.046093,0.139478,0.071424,0.016710,0.042377,0.076187,0.071979
1,0.228361,0.010791,0.116364,0.144383,0.500294,0.156641,0.536311,0.259939,0.417631,0.546701
2,0.012849,0.000000,0.029091,0.055861,0.000000,0.002577,0.289846,0.117519,0.251695,0.232219
3,0.047651,0.000000,0.005455,0.041209,0.079470,0.085530,0.048843,0.128877,0.185082,0.124250
4,0.108380,0.017986,0.063636,0.098291,0.022701,0.033717,0.043380,0.182176,0.218588,0.132819
...,...,...,...,...,...,...,...,...,...,...
67,0.026335,0.000000,0.052727,0.210317,0.026637,0.099273,0.023458,0.247270,0.264858,0.134533
68,0.089230,0.000000,0.094545,0.166056,0.011884,0.049955,0.049486,0.168196,0.268049,0.197943
69,0.169265,0.000000,0.360000,0.115079,0.108572,0.311414,0.227828,0.266929,0.336657,0.409597
70,0.016738,0.000000,0.143636,0.227411,0.011001,0.012924,0.022815,0.076453,0.115676,0.051414


## Regresión lineal

Para cada uno de los 3 métodos de selección de atributos se realizó una regresión para identificar cual de los tres métodos proporciona mejores resultados.

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

metodos=['Selección de atributos usando filtro','Selección de atributos usando selección de atributos recursiva','Selección de atributos secuencial']
data=[X_filtrada_f,X_filtrada_RFE,X_filtrada_SFE]



Para la tabla 3

In [29]:
from sklearn.linear_model import LinearRegression
modelo=LinearRegression()
for i in range(0,len(metodos)):
  X_train, X_test, y_train, y_test = train_test_split(data[i], y, test_size=0.20, random_state=42)
  modelo.fit(X_train, y_train)
  y_pred = modelo.predict(X_test)
  rmse = mean_squared_error(y_test, y_pred,squared=False)
  r2 = r2_score(y_test, y_pred)
  print('Para {}'.format(metodos[i]))
  print('Se obtuvo un modelo con las siguientes características:')
  print(" Error cuadrático medio (RMSE) :", rmse)
  print("Coeficiente de determinación:", r2)
  print('--------------------------------------')

Para Selección de atributos usando filtro
Se obtuvo un modelo con las siguientes características:
 Error cuadrático medio (RMSE) : 0.21065692992998597
Coeficiente de determinación: -0.07098138194297765
--------------------------------------
Para Selección de atributos usando selección de atributos recursiva
Se obtuvo un modelo con las siguientes características:
 Error cuadrático medio (RMSE) : 0.09282403627716587
Coeficiente de determinación: 0.7920536423003284
--------------------------------------
Para Selección de atributos secuencial
Se obtuvo un modelo con las siguientes características:
 Error cuadrático medio (RMSE) : 0.08155607617078187
Coeficiente de determinación: 0.8394748569707463
--------------------------------------


In [30]:
modelo = LinearRegression()
variables_importantes_por_metodo = []

for i in range(len(metodos)):
    X_train, X_test, y_train, y_test = train_test_split(data[i], y, test_size=0.20, random_state=42)
    modelo.fit(X_train, y_train)
    coeficientes = modelo.coef_
    variables_importantes_por_metodo.append(coeficientes)

# Los coeficientes de cada variable por cada método
for i, coeficientes in enumerate(variables_importantes_por_metodo):
    print(f'Variables importantes para el método {metodos[i]}:')
    for j, coeficiente in enumerate(coeficientes):
        print(f'Variable {j + 1}: {coeficiente}')
    print('--------------------------------------')


Variables importantes para el método Selección de atributos usando filtro:
Variable 1: 0.4071180794368292
Variable 2: -1.6774502985457602
Variable 3: 0.9760437628776033
Variable 4: 1.2559056140095086
Variable 5: 0.061087941923889943
Variable 6: 1.7081144355936133
Variable 7: -0.5110029337739712
Variable 8: -0.6547027329014946
Variable 9: 0.46815275307769605
Variable 10: 0.8653281797722872
--------------------------------------
Variables importantes para el método Selección de atributos usando selección de atributos recursiva:
Variable 1: 0.48315221510252265
Variable 2: -2.3537055882390017
Variable 3: -2.4254237820233615
Variable 4: -0.790114666044712
Variable 5: 0.6881827630795514
Variable 6: -2.0722324111128185
Variable 7: 2.297097555017694
Variable 8: 1.2780066031288373
Variable 9: -1.3529523671167292
Variable 10: 1.0804099163373455
--------------------------------------
Variables importantes para el método Selección de atributos secuencial:
Variable 1: -0.4792819975647401
Variable 2